In [1]:

from fastapi import APIRouter
from supabase import create_client, Client
from config import SUPABASE_URL, SUPABASE_KEY, PODCAST_API_KEY, PODCAST_USER_ID, PODCAST_API_URL
import requests

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [2]:
sample_headers = {
    "Content-Type": "application/json",
    "X-USER-ID": PODCAST_USER_ID,
    "X-API-KEY": PODCAST_API_KEY
}

In [18]:
def get_podcast_data(podcast_name = "Lex Fridman"):
    podcast_request_body = f"""{{
        getPodcastSeries(name: "Huberman Lab") {{
        uuid
        name
        authorName
        imageUrl
        description
        episodes(page: 1, limitPerPage: 10) {{
            uuid
            datePublished
            name
            description
            duration
            imageUrl
            audioUrl
        }}
        genres
    }}
    }}
    """
    json_request = {"query": podcast_request_body}
    res = requests.post(PODCAST_API_URL, headers=sample_headers, json=json_request)
    return res.json()

In [19]:
res = get_podcast_data()

In [20]:
res['data']['getPodcastSeries']['episodes']

[{'uuid': 'deff7085-bd0b-4119-be10-5e20af84e2dc',
  'datePublished': 1725868800,
  'name': 'Dr. Marc Brackett: How to Increase Your Emotional Intelligence',
  'description': '<p>In this episode, my guest is <a href="https://marcbrackett.com/">Dr. Marc Brackett, Ph.D.</a>, a professor in the Child Study Center at Yale University, director of the Yale Center for Emotional Intelligence, and a world expert on what emotions are, how to interpret them, and how to work with emotions to yield a more impactful, meaningful and healthier life. We explore differences between introverts and extroverts, in-person and text-based emotional communication, and how emotional suppression impacts us.</p><p>We discuss emotional intelligence and describe tools to improve emotional regulation and communication in personal and professional relationships. We also explore the role of emotions in learning, resolving conflicts, and decision-making. We also discuss bullying in kids and adults, both in person and on

In [21]:
image_urls = [{'image_url': episode['imageUrl'], "name": episode["name"]} for episode in res['data']['getPodcastSeries']['episodes']]
image_urls

[{'image_url': None,
  'name': 'Dr. Marc Brackett: How to Increase Your Emotional Intelligence'},
 {'image_url': None,
  'name': 'Dr. Jamil Zaki: How to Cultivate a Positive, Growth-Oriented Mindset'},
 {'image_url': None, 'name': 'Optimal Protocols for Studying & Learning'},
 {'image_url': None,
  'name': 'Dr. Teo Soleymani: How to Improve & Protect Your Skin Health & Appearance'},
 {'image_url': None,
  'name': 'Perform with Dr. Andy Galpin: Nutrition to Support Brain Health & Offset Brain Injuries'},
 {'image_url': None,
  'name': 'Dr. Layne Norton: Tools for Nutrition & Fitness'},
 {'image_url': None,
  'name': 'Dr. Martha Beck: Access Your Best Self With Mind-Body Practices, Belief Testing & Imagination'},
 {'image_url': 'https://megaphone.imgix.net/podcasts/1d3d1680-5037-11ef-b39f-bf5e480148f7/image/34256f63c8c1fc50fc11ffcb1d5265af.jpg?ixlib=rails-4.3.1&max-w=3000&max-h=3000&fit=crop&auto=format,compress',
  'name': 'AMA #18: Cold Therapy Advice, Skin Health Tips, Motivation, Lea

In [51]:
from datetime import datetime

def insert_podcast(data):
    return (
        supabase.table("podcasts").insert({
        "title": data['name'],
        "author_name": data['authorName'],
        "image_url": data['imageUrl'],
        "description": data['description'],
    }).execute())

def insert_episodes(data):
    return (supabase.table("episodes").insert([{
        "title": episode['name'],
        "description": episode['description'],
        "duration": episode['duration'],
        "created_at": datetime.fromtimestamp(episode['datePublished']).isoformat(),
        "audio_url": episode['audioUrl'],
    } for episode in data]).execute())

In [52]:
response = insert_podcast(res['data']['getPodcastSeries'])

In [53]:
episode_response = insert_episodes(res['data']['getPodcastSeries']['episodes'])

In [54]:
episode_ids =[episode['id'] for episode in episode_response.data]
episode_ids

[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

In [55]:
podcast_id = response.data[0]['id']
podcast_id

6

In [56]:
def insert_podcast_episodes(podcast_id, episode_ids):
    return (supabase.table("episode_podcast").insert([{
        "podcast_id": podcast_id,
        "episode_id": episode_id
    } for episode_id in episode_ids]).execute())

In [57]:
relationship_response = insert_podcast_episodes(podcast_id, episode_ids)